In [1]:
import os
import cntk as ct

from src.ferplus import FERPlusReader, FERPlusParameters
from src.models import *

In [2]:
def cost_func(training_mode, prediction, target):
    '''
    We use cross entropy in most mode, except for the multi-label mode, which require treating
    multiple labels exactly the same.
    '''
    train_loss = None
    if training_mode == 'majority' or training_mode == 'probability' or training_mode == 'crossentropy': 
        # Cross Entropy.
        train_loss = ct.negate(ct.reduce_sum(ct.element_times(target, ct.log(prediction)), axis=-1))
    elif training_mode == 'multi_target':
        train_loss = ct.negate(ct.log(ct.reduce_max(ct.element_times(target, prediction), axis=-1)))

    return train_loss

In [3]:
# maior valor seta para 1, demais seta 0
def binarizar_array(array):
    max_index = np.argmax(array)  
    binarizado = np.zeros_like(array)  
    binarizado[max_index] = 1
    return binarizado

In [4]:
## parametros
model_name='VGG13'

base_folder = 'data'
test_folders  = ['FER2013Test']

# training_mode, best_epoch = ("majority",80)
# training_mode, best_epoch = ("probability",78)
# training_mode, best_epoch = ("crossentropy",98)
training_mode, best_epoch = ("multi_target",89)

In [5]:
## folders models
output_model_path   = os.path.join(base_folder, R'models')
output_model_folder = os.path.join(output_model_path, model_name + '_' + training_mode)

if not os.path.exists(output_model_folder):
    os.makedirs(output_model_folder)

In [6]:
## folders tests
output_test_path   = os.path.join(base_folder, R'tests')
output_test_folder = os.path.join(output_test_path, model_name + '_' + training_mode)

if not os.path.exists(output_test_folder):
    os.makedirs(output_test_folder)

In [7]:
emotion_table = {'neutral'  : 0, 
                 'happiness': 1, 
                 'surprise' : 2, 
                 'sadness'  : 3, 
                 'anger'    : 4, 
                 'disgust'  : 5, 
                 'fear'     : 6, 
                 'contempt' : 7}

emotion_labels = sorted(emotion_table, key=emotion_table.get)

num_classes = len(emotion_table)

In [8]:
## leitura do modelo
model = build_model(num_classes, model_name)

In [9]:
## set the input variables.
input_var = ct.input((1, model.input_height, model.input_width), np.float32)
label_var = ct.input((num_classes), np.float32)

In [10]:
## training_mode interfere nos labels, fazendo com que seja 0 e 1 ou entre 0 e 1.
## por algum motivo, no test_and_val_params ele está estatico em 'majatory' !! DESCOBRIR MOTIVO
## usando training_mode diferente de majority, y_true deve ser binarizado igual y_pred ( transformei max em 1 e o resto em zero)

# test_and_val_params = FERPlusParameters(num_classes, model.input_height, model.input_width, "majority", determinisitc=True, shuffle=False)
test_and_val_params = FERPlusParameters(num_classes, model.input_height, model.input_width, training_mode, determinisitc=True, shuffle=False)
test_data_reader = FERPlusReader.create(base_folder, test_folders, "label.csv", test_and_val_params)

In [11]:
epoch_size = test_data_reader.size()
minibatch_size = 32

In [12]:
# get the probalistic output of the model.
z    = model.model(input_var)
pred = ct.softmax(z)

In [13]:
# Training config
lr_per_minibatch       = [model.learning_rate]*20 + [model.learning_rate / 2.0]*20 + [model.learning_rate / 10.0]
mm_time_constant       = -minibatch_size/np.log(0.9)
lr_schedule            = ct.learning_rate_schedule(lr_per_minibatch, unit=ct.UnitType.minibatch, epoch_size=epoch_size)
mm_schedule            = ct.momentum_as_time_constant_schedule(mm_time_constant)

# loss and error cost
train_loss = cost_func(training_mode, pred, label_var)
pe         = ct.classification_error(z, label_var)

In [14]:
# construct the trainer
learner = ct.momentum_sgd(z.parameters, lr_schedule, mm_schedule)
trainer = ct.Trainer(z, (train_loss, pe), learner)
trainer.total_number_of_samples_seen

0

In [15]:
print(os.path.join(output_model_folder, "model_{}".format(best_epoch)))
trainer.restore_from_checkpoint(os.path.join(output_model_folder, "model_{}".format(best_epoch)))
trainer.total_number_of_samples_seen

data/models/VGG13_multi_target/model_89


2490030

In [16]:
y_true_batch = []
y_pred_batch = []

while test_data_reader.has_more():
    images, labels, current_batch_size = test_data_reader.next_minibatch(minibatch_size)
    y_true_batch.append(labels)
    y_pred_batch.append(trainer.model.eval({input_var: images}))

In [17]:
y_true = np.concatenate(y_true_batch, axis=0)
y_true.shape

(3466, 8)

In [18]:
y_pred = np.concatenate(y_pred_batch, axis=0)
y_pred.shape

(3466, 8)

In [19]:
y_true

array([[ 0.001,  0.001,  0.001, ...,  0.001,  0.001,  1.   ],
       [ 1.   ,  0.001,  0.001, ...,  0.001,  0.001,  0.001],
       [ 1.   ,  0.001,  0.001, ...,  0.001,  0.001,  0.001],
       ..., 
       [ 0.001,  0.001,  0.001, ...,  0.001,  0.001,  0.001],
       [ 0.001,  1.   ,  0.001, ...,  0.001,  0.001,  0.001],
       [ 1.   ,  0.001,  0.001, ...,  0.001,  0.001,  0.001]], dtype=float32)

In [20]:
y_true_bin = np.array([ binarizar_array(arr) for arr in y_true ])
y_true_bin

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [21]:
y_pred

array([[  6.55176115,  -4.96301508,  -0.84649754, ...,  -1.88809967,
         -3.55828691,   1.69767296],
       [  8.81030273,  -3.67780232,  -3.33396411, ...,  -3.07839966,
         -6.03207159,   0.80377328],
       [  7.82557154,  -4.08956671,   0.24173436, ...,  -3.77943754,
         -4.00930405,   0.87448514],
       ..., 
       [  4.05337334,  -1.37536943,   0.64318717, ...,  -1.74678755,
         -2.04518104,  -0.75132573],
       [ -0.90338624,  12.91828537,  -0.38926336, ...,  -4.92607498,
         -3.65607572,  -4.16153336],
       [  7.26770306,  -6.34903049,  -4.62837315, ...,  -1.13939333,
         -4.59666681,   2.85757732]], dtype=float32)

In [22]:
y_pred_bin = np.array([ binarizar_array(arr) for arr in y_pred ])
y_pred_bin

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [23]:
import sklearn.metrics as m

y_true=y_true_bin
y_pred=y_pred_bin

print('ACC: ',m.accuracy_score(y_true, y_pred))
print(m.classification_report(y_true, y_pred, target_names=emotion_labels))
print('cohen-kappa: ',m.cohen_kappa_score(y_true.argmax(axis=1), y_pred.argmax(axis=1), labels=sorted(emotion_table.values())))
print('cm:\n', m.confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1)))

ACC:  0.843046739758
             precision    recall  f1-score   support

    neutral       0.90      0.85      0.88      1448
  happiness       0.92      0.92      0.92       925
   surprise       0.79      0.88      0.83       431
    sadness       0.63      0.65      0.64       322
      anger       0.71      0.82      0.76       261
    disgust       0.46      0.50      0.48        12
       fear       0.61      0.56      0.58        54
   contempt       0.00      0.00      0.00        13

avg / total       0.84      0.84      0.84      3466

cohen-kappa:  0.785315912856
cm:
 [[1236   48   43   91   27    1    2    0]
 [  19  848   30   14   10    1    2    1]
 [  12    7  379    5   21    0    7    0]
 [  66   13    4  209   23    2    5    0]
 [  20    7    9    5  214    3    3    0]
 [   3    0    0    0    3    6    0    0]
 [   4    1   13    5    1    0   30    0]
 [   8    1    0    2    2    0    0    0]]


In [24]:
print(os.path.join(output_test_folder, "test_{}.npz".format(best_epoch)))
np.savez_compressed(os.path.join(output_test_folder, "test_{}.npz".format(best_epoch)), y_true=y_true_bin, y_pred=y_pred_bin)

data/tests/VGG13_multi_target/test_89.npz
